# Packages

In [0]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
import requests
from pandas.io.json import json_normalize

In [0]:
!pip install webweb

    100% |████████████████████████████████| 204kB 24.3MB/s 
  Stored in directory: /root/.cache/pip/wheels/a5/07/e1/9bbaa0fefadd9710b532db01158bfda19a0223ed23ce7d5314
Successfully built webweb


# List of documents

In [0]:
_FMlinkList = {
    'october2017':"http://educatorinnovator.org/how-young-activists-deploy-digital-tools-for-social-change",
    'november2017':"https://educatorinnovator.org/wp-content/uploads/2017/10/RRE_Chapter-6_Civic-Participation-Remiagined_0091732X17690121.pdf",
    'december2017':"https://educatorinnovator.org/wp-content/uploads/2017/10/Critical-Literacy-And-Our-Students-Lives.pdf",
    'january2018':"https://educatorinnovator.org/wp-content/uploads/2017/12/OurDeclaration_PG31-35.pdf",
    'february2018':"https://educatorinnovator.org/wp-content/uploads/2018/02/Educating-for-Democracy-in-a-Partisan-Age.pdf.pdf",
    'march2018':"https://educatorinnovator.org/wp-content/uploads/2018/02/The-Stories-They-Tell-.pdf",
    'april2018':"https://educatorinnovator.org/wp-content/uploads/2018/03/Educating-Youth-for-Online-Civic-and-Political-Dialogue_-A-Conceptual-Framework-for-the-Digital-Age-_-Journal-of-Digital-and-Media-Literacy.pdf",
    'may2018':"https://educatorinnovator.org/wp-content/uploads/2018/03/zemelman_websample.pdf",
    'june2018':"http://www.ncte.org/library/NCTEFiles/Resources/Journals/VM/0254-may2018/VM0254Leading.pdf"
}

# Helper functions

In [0]:
# Build dataframe from hypothes.is API
def H_API(session,uri):
    n = 0
    payload = {'url':uri}
    r = requests.get('https://hypothes.is/api/search',params=payload)
    tmp = pd.DataFrame(columns=['created', 'document.title', 'group', 'id', 'links.html',
       'links.incontext', 'links.json', 'permissions.admin',
       'permissions.delete', 'permissions.read', 'permissions.update',
       'references', 'tags', 'target', 'text', 'updated', 'uri', 'user'])
    total = r.json()['total']
    while n < total:
        #print(n)
        tmp = tmp.append(json_normalize(r.json()['rows']))
        n = len(tmp)
        payload = {
            'url':uri,
            'offset':n}
        r = requests.get('https://hypothes.is/api/search',params=payload)
    tmp['session'] = session
    tmp['updated'] = pd.to_datetime(tmp['updated'])
    tmp['created'] = pd.to_datetime(tmp['created'])
    tmp = tmp.set_index(tmp['created'])
    return tmp

#returns the number of times a message was in a reference list of other messages
def countReplies(msgId):
    return len(df.dropna()[df['references'].dropna().map(lambda x: msgId in x)])
def listOfIds(msgId):
    return df.dropna()[df['references'].dropna().map(lambda x: msgId in x)]['id'].values
def replyTimeDelta(x):
    if len(df[df['id']==x]['created']) == 0:
      return None
    else:
      return df[df['id']==x]['created'].iloc[0]
    
#Builds a reply chain of users
def replyChain(refs):
  replychain = []
  for i in refs:
    if len(df[df['id']==i]['user']) == 0:
      return None
    replychain.append(df[df['id']==i]['user'].iloc[0])
  return replychain

# Create dataFrames

In [0]:
threadsByMonth = {}
for session,link in _FMlinkList.items():    
  #Build network of threads
  df = pd.DataFrame(columns=['created','document.title','group','id','links.html',
         'links.incontext','links.json','permissions.admin',
         'permissions.delete','permissions.read','permissions.update',
         'references','tags','target','text','updated','uri','user','session'])
  df = df.append(H_API(session,link))
  df = df.tz_localize('UTC').tz_convert('US/Mountain')

  #Modify df for network analysis
  df = df[['created','id','references','session','tags','target', 'text','user']]
  df['replyTo'] = df['references'].map(lambda x: x[-1],na_action='ignore')
  df['user'] = df['user'].map(lambda x: x[5:-12])

  #Time betweeen message and reply
  df['replyDelay'] = df[df['replyTo'].notnull()]['created']\
  - df[df['replyTo'].notnull()]['replyTo'].map(lambda x: replyTimeDelta(x))

  #convert to minutes
  df['replyDelay'] = df['replyDelay'].map(lambda x: x.total_seconds() / 60)
  #builds reply chain of users
  df['replyChain'] = df[df['references'].notnull()]['references'].map(lambda x: replyChain(x),na_action='ignore')

  G = nx.from_pandas_edgelist(df[df['references'].notnull()],'id','replyTo'
                             ,edge_attr=['tags','target','text','user','replyDelay']
                             ,create_using=nx.DiGraph())

  # Build dataframe of threads with list of ids per thread
  threads = []
  for x in nx.connected_components(nx.to_undirected(G)):
      threads.append(x)

  # list of text per thread
  textT = []
  timesT = []
  usersT = []
  repliesT = []
  for thread in threads:    
      T = []
      t = []
      u = []
      r = []
      for key,val in df[df['id'].isin(thread)].iterrows():
          T.append(val['text'])
          t.append(val['replyDelay'])
          u.append(val['user'])
          r.append(val['replyChain'])
      textT.append(T)
      timesT.append(t)
      usersT.append(u)
      repliesT.append(r)

  #build threads dataframe
  dfThreads = pd.DataFrame(pd.Series(data=threads),columns=['ids'])
  dfThreads['texts'] = pd.Series(textT)
  dfThreads['timeDelays'] = pd.Series(timesT)
  dfThreads['users'] = pd.Series(usersT)
  dfThreads['replyChains'] = pd.Series(repliesT)
  dfThreads['totalMsgs'] = dfThreads['ids'].map(lambda x: len(x))
  dfThreads = dfThreads.sort_values(by='totalMsgs',ascending=False)
  dfThreads.reset_index(drop=True,inplace=True)
  
  threadsByMonth[session] = dfThreads

##Threads with 3 or more messages 
###Prints text by participants

In [0]:
for k in threadsByMonth:
  print("****************************************************************************************************")
  print("****************************************************************************************************")
  print('                                 Document name:',k)
  print('              URL:',_FMlinkList[k])
  print("****************************************************************************************************")
  print("****************************************************************************************************")
  for key,vals in threadsByMonth[k].iterrows():
    if (vals['totalMsgs'] < 3): continue
    print("**************************************************")
    print('NEW THREAD')
    print('Message Count: ',vals['totalMsgs'])
    print("**************************************************")
    ul = vals['users']
    tl = vals['texts']
    rl = vals['replyChains']
    ul.reverse()
    tl.reverse()
    rl.reverse()
    for p in zip(rl, ul, tl):
      if (str(p[0]) == 'nan'):
        print(p[1]+":",p[2])
      else: print(str(p[0]),p[1]+":",p[2])
      print("-------------------")
    print()

In [0]:
G.graph

{}

In [0]:
for x in nx.connected_components(nx.to_undirected(G)):
  print(x)
  #threads.append(x)
  #nx.draw(x, with_labels=True, font_weight='bold')

{'rjsQ7GaHEeiMCCPSia1NEg', 's0u-fGZSEeiPqc_LN9yVyw', 'QfIICGdCEeiicdt7z8tmYg', '5s6sOtLsEeisBesXFxE2cw', 'F96S9GjzEeiiJCNcd4OFIQ'}
{'kSuxSM-jEeitPI-ivMkXCg', 'GbdHZM_nEeitGK-suWUScw'}
{'wmAccM_mEeibTxOwSjpDKQ', 'qtVgKs-kEeiFHO9HXMoyRA'}
{'NAqDXM_mEei_LE9W8Wt0HQ', 'zpg4sM-vEeiubRvJC1Fc5A'}
{'jnEMgM-5Eei3u-_FGkxHww', 'qTIeAs_lEeit47NKat9c3Q'}
{'18cE0M-kEeiaVzPJw_siVA', 'zY9baG92Eei91AMgtQzLPQ'}
{'qPAvmJtpEeiO3u-io0-SiQ', 'iuj8zmmaEeixi3PKXBLbqw', 'N4nGgJsiEeiFYC9nWRYn6A', 'WEjI2mZIEeicWztGkmYDRw', '538E9GbMEei8Iv-jCcdqTg'}
{'3CX-hmhYEeii1dvrZqK-_w', 'ceFbAGipEeiurQuLdztQYQ', '0OIVmpKJEeiMBnMsrlUrNg'}
{'mWMkepKIEeioOG-ZMZBEgA', 'mMil_ohQEeicUFO3Mh_WeQ'}
{'1UFogGZGEeib1sMyNoWQgQ', 'eEf86JKIEeioN0c6YnE7uQ'}
{'0YcEnmhXEei15Deeq8s4Hw', 'A4J3qIhREeiyBqebVJCzwA', 'wykwlGimEei5SR9_o7hyOg'}
{'jww6OGj5EeiYbENLnyh70Q', 'rQcK0mlwEeiu3xPYAh8PAQ', '3EnmZGhzEeij9Eeoq6rh9A', 'QTsmwGhPEeikw6P6gIp9ag', 'ha-_TGy4Eei7AiemkeiHYQ', 'RCdssGi2Eei5UPsczZj0BQ', 'CYvVNGZSEeiz-EMid5eGsA', 'CajSNGbOEeich1O72g03vQ', 